In [2]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 16.7 MB/s 


In [3]:
# Import Module
import pandas as pd
import numpy as np
import missingno as msno
import re
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import nltk
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.corpus import stopwords
from gensim.models import Word2Vec, FastText
import nltk.stem as stemmer

In [4]:
# Path file
path="/content/Data (1).csv"

In [5]:
# Open File
data = pd.read_csv(path)

In [6]:
# Check File
data

,bully,tweet,individual,group,gender,physical,race,religion
0,no,USER terimakasih Ustadz sudah bersuara tentang...,0,0,0,0,0,0
1,no,USER USER Maaf sebenarnya twiter pertama kali ...,0,0,0,0,0,0
2,yes,USER Anjing tai goblok idiot bangsat monyet ba...,1,0,1,1,0,0
3,no,"Hadiri Lokakarya Kebudayaan Daerah, Bupati Rup...",0,0,0,0,0,0
4,yes,USER USER USER yg kaya gini layak di tangkap.,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
10530,yes,USER USER USER kyak dah lahir aja ngomong soeh...,0,1,0,0,0,0
10531,yes,USER USER USER USER USER Pencitraan Lu...ah......,1,0,0,0,0,0
10532,yes,USER USER USER USER USER cebong doang yg tukan...,0,1,0,0,0,0
10533,yes,USER BIASA LAH PROF KAUM KAMBING BENGEK YA BEG...,0,1,0,0,0,0


# Lowercase dll

In [7]:
# Cleaning text function
def remove_punct(text) :
    text = re.sub(r'[^a-zA-Z0-9]'," ",str(text))
    text = re.sub(r'\b\w(1,2)\b',"",text)
    text = re.sub(r"\s\s+"," ",text)
    text = re.sub("\d+", "", text)
    text = text.lower()
    text = text.strip(" ")
    return text

In [8]:
# Apply the function
data["hasil"] = data["tweet"].apply(lambda x: remove_punct(x))

# Tokenizing

In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
# Tokenizing 
data['tokenize_hasil'] = data.apply(lambda row: nltk.word_tokenize(row['hasil']), axis=1)

In [13]:
# Check result
data

,bully,tweet,individual,group,gender,physical,race,religion,hasil,tokenize_hasil
0,no,USER terimakasih Ustadz sudah bersuara tentang...,0,0,0,0,0,0,user terimakasih ustadz sudah bersuara tentang...,"[user, terimakasih, ustadz, sudah, bersuara, t..."
1,no,USER USER Maaf sebenarnya twiter pertama kali ...,0,0,0,0,0,0,user user maaf sebenarnya twiter pertama kali ...,"[user, user, maaf, sebenarnya, twiter, pertama..."
2,yes,USER Anjing tai goblok idiot bangsat monyet ba...,1,0,1,1,0,0,user anjing tai goblok idiot bangsat monyet ba...,"[user, anjing, tai, goblok, idiot, bangsat, mo..."
3,no,"Hadiri Lokakarya Kebudayaan Daerah, Bupati Rup...",0,0,0,0,0,0,hadiri lokakarya kebudayaan daerah bupati rupi...,"[hadiri, lokakarya, kebudayaan, daerah, bupati..."
4,yes,USER USER USER yg kaya gini layak di tangkap.,1,0,0,0,0,0,user user user yg kaya gini layak di tangkap,"[user, user, user, yg, kaya, gini, layak, di, ..."
...,...,...,...,...,...,...,...,...,...,...
10530,yes,USER USER USER kyak dah lahir aja ngomong soeh...,0,1,0,0,0,0,user user user kyak dah lahir aja ngomong soeh...,"[user, user, user, kyak, dah, lahir, aja, ngom..."
10531,yes,USER USER USER USER USER Pencitraan Lu...ah......,1,0,0,0,0,0,user user user user user pencitraan lu ah prab...,"[user, user, user, user, user, pencitraan, lu,..."
10532,yes,USER USER USER USER USER cebong doang yg tukan...,0,1,0,0,0,0,user user user user user cebong doang yg tukan...,"[user, user, user, user, user, cebong, doang, ..."
10533,yes,USER BIASA LAH PROF KAUM KAMBING BENGEK YA BEG...,0,1,0,0,0,0,user biasa lah prof kaum kambing bengek ya beg...,"[user, biasa, lah, prof, kaum, kambing, bengek..."


# Stopword Removal

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
# Define Indonesia stopword
stopword = stopwords.words('indonesian')
extra = ["user", "rt", "jadi", "yg", "url", "dgn", "lu", "lo", "di"]
stopword = stopword + extra 

In [19]:
# Stopword removal function
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

In [20]:
# Apply the fuction
data["STOP_REMOVAL"]=data["tokenize_hasil"].apply(lambda x: remove_stopwords(x))

In [21]:
# Check result
data

,bully,tweet,individual,group,gender,physical,race,religion,hasil,tokenize_hasil,STOP_REMOVAL
0,no,USER terimakasih Ustadz sudah bersuara tentang...,0,0,0,0,0,0,user terimakasih ustadz sudah bersuara tentang...,"[user, terimakasih, ustadz, sudah, bersuara, t...","[terimakasih, ustadz, bersuara, radikal, radik..."
1,no,USER USER Maaf sebenarnya twiter pertama kali ...,0,0,0,0,0,0,user user maaf sebenarnya twiter pertama kali ...,"[user, user, maaf, sebenarnya, twiter, pertama...","[maaf, twiter, kali, dbuat, orang, bego]"
2,yes,USER Anjing tai goblok idiot bangsat monyet ba...,1,0,1,1,0,0,user anjing tai goblok idiot bangsat monyet ba...,"[user, anjing, tai, goblok, idiot, bangsat, mo...","[anjing, tai, goblok, idiot, bangsat, monyet, ..."
3,no,"Hadiri Lokakarya Kebudayaan Daerah, Bupati Rup...",0,0,0,0,0,0,hadiri lokakarya kebudayaan daerah bupati rupi...,"[hadiri, lokakarya, kebudayaan, daerah, bupati...","[hadiri, lokakarya, kebudayaan, daerah, bupati..."
4,yes,USER USER USER yg kaya gini layak di tangkap.,1,0,0,0,0,0,user user user yg kaya gini layak di tangkap,"[user, user, user, yg, kaya, gini, layak, di, ...","[kaya, gini, layak, tangkap]"
...,...,...,...,...,...,...,...,...,...,...,...
10530,yes,USER USER USER kyak dah lahir aja ngomong soeh...,0,1,0,0,0,0,user user user kyak dah lahir aja ngomong soeh...,"[user, user, user, kyak, dah, lahir, aja, ngom...","[kyak, dah, lahir, aja, ngomong, soeharto, l, ..."
10531,yes,USER USER USER USER USER Pencitraan Lu...ah......,1,0,0,0,0,0,user user user user user pencitraan lu ah prab...,"[user, user, user, user, user, pencitraan, lu,...","[pencitraan, ah, prabowo, msh, tidur]"
10532,yes,USER USER USER USER USER cebong doang yg tukan...,0,1,0,0,0,0,user user user user user cebong doang yg tukan...,"[user, user, user, user, user, cebong, doang, ...","[cebong, doang, tukang, ngamuk]"
10533,yes,USER BIASA LAH PROF KAUM KAMBING BENGEK YA BEG...,0,1,0,0,0,0,user biasa lah prof kaum kambing bengek ya beg...,"[user, biasa, lah, prof, kaum, kambing, bengek...","[prof, kaum, kambing, bengek, ya, beda, dikit,..."


# Stemming

In [22]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [23]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [25]:
# stemmed 
def stemming(review):
    do = []
    for w in review:
        dt=stemmer.stem(w)
        do.append(dt)
    d_clean=[]
    d_clean=" ".join(do)
    print(d_clean)
    return d_clean


In [26]:
data["clean"]=data["STOP_REMOVAL"].apply(stemming)

Output streaming akan dipotong hingga 5000 baris terakhir.
biadab uus keparat tunggu dimonas cuman koar koar hidup anjing
media indonesia cina komunis
insyaallah mudah presiden pro rakyat cinta rakyat rakyat aseng
swasta katolik yoi tk sampe sma swasta katolik ketemu temen aja mikir yaudahlah situ
nahdliyin setop hina maruf amin
ngeri banget ati ati nek doa moga depok aman moga aja isi bom beneran moga ga ya allah puasa gin bikin ga tenang aja si ya
edan
al habib abdullah bin mukshin al attas keramat bogor orang buta orang banyak harta orang buta orang ilmu agama
anak pki pki
udik takjub gituuu gileeee canggih ye mobil asa naek kit reder knight rider starring david haselhoff ngegas ngerem
jokowi motor listrik produsen motor langsung lirik papua presiden joko w
tolol emang kelompok anjing
sabtu pagi masyarakat adat baduy lanjut jalan kantor gubernur banten kawasan pusat perintah provinsi banten kpb curug kota serang sebabaduy dm
kasar komunis free sex klo cadang minyak limpah mudah capa

In [27]:
data

,bully,tweet,individual,group,gender,physical,race,religion,hasil,tokenize_hasil,STOP_REMOVAL,clean
0,no,USER terimakasih Ustadz sudah bersuara tentang...,0,0,0,0,0,0,user terimakasih ustadz sudah bersuara tentang...,"[user, terimakasih, ustadz, sudah, bersuara, t...","[terimakasih, ustadz, bersuara, radikal, radik...",terimakasih ustadz suara radikal radikal pikir...
1,no,USER USER Maaf sebenarnya twiter pertama kali ...,0,0,0,0,0,0,user user maaf sebenarnya twiter pertama kali ...,"[user, user, maaf, sebenarnya, twiter, pertama...","[maaf, twiter, kali, dbuat, orang, bego]",maaf twiter kali dbuat orang bego
2,yes,USER Anjing tai goblok idiot bangsat monyet ba...,1,0,1,1,0,0,user anjing tai goblok idiot bangsat monyet ba...,"[user, anjing, tai, goblok, idiot, bangsat, mo...","[anjing, tai, goblok, idiot, bangsat, monyet, ...",anjing tai goblok idiot bangsat monyet babi fu...
3,no,"Hadiri Lokakarya Kebudayaan Daerah, Bupati Rup...",0,0,0,0,0,0,hadiri lokakarya kebudayaan daerah bupati rupi...,"[hadiri, lokakarya, kebudayaan, daerah, bupati...","[hadiri, lokakarya, kebudayaan, daerah, bupati...",hadir lokakarya budaya daerah bupati rupinus a...
4,yes,USER USER USER yg kaya gini layak di tangkap.,1,0,0,0,0,0,user user user yg kaya gini layak di tangkap,"[user, user, user, yg, kaya, gini, layak, di, ...","[kaya, gini, layak, tangkap]",kaya gin layak tangkap
...,...,...,...,...,...,...,...,...,...,...,...,...
10530,yes,USER USER USER kyak dah lahir aja ngomong soeh...,0,1,0,0,0,0,user user user kyak dah lahir aja ngomong soeh...,"[user, user, user, kyak, dah, lahir, aja, ngom...","[kyak, dah, lahir, aja, ngomong, soeharto, l, ...",kyak dah lahir aja ngomong soeharto l ahli ceb...
10531,yes,USER USER USER USER USER Pencitraan Lu...ah......,1,0,0,0,0,0,user user user user user pencitraan lu ah prab...,"[user, user, user, user, user, pencitraan, lu,...","[pencitraan, ah, prabowo, msh, tidur]",citra ah prabowo msh tidur
10532,yes,USER USER USER USER USER cebong doang yg tukan...,0,1,0,0,0,0,user user user user user cebong doang yg tukan...,"[user, user, user, user, user, cebong, doang, ...","[cebong, doang, tukang, ngamuk]",cebong doang tukang ngamuk
10533,yes,USER BIASA LAH PROF KAUM KAMBING BENGEK YA BEG...,0,1,0,0,0,0,user biasa lah prof kaum kambing bengek ya beg...,"[user, biasa, lah, prof, kaum, kambing, bengek...","[prof, kaum, kambing, bengek, ya, beda, dikit,...",prof kaum kambing bengek ya beda dikit bilang ...


In [29]:
# Export to excel
data.to_csv("preprocessed_data_zein.csv", index = False)  